In [ ]:
!pip install requests

In [51]:
import requests
import datetime
from urllib.parse import urlencode

In [52]:
import base64

In [53]:
client_id = 'CLIENT ID'
client_secret = 'CLIENT SECRET ID'

In [54]:
#import datetime
class SpotifyAPI(object):
  access_token = None
  access_token_expires = datetime.datetime.now()
  access_token_did_expire = True
  client_id = None
  client_secret = None
  token_url="https://accounts.spotify.com/api/token"
  

  def __init__(self, client_id, client_secret, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.client_id = client_id
    self.client_secret = client_secret

  def get_client_credentials(self):
    """
    Returns a base64 encoded string
    """
    client_id = self.client_id
    client_secret = self.client_secret
    if client_id == None or client_secret == None:
      raise Exception("You must set client_id and client_secret")
    client_creds = f"{client_id}:{client_secret}"
    client_creds_b64 = base64.b64encode(client_creds.encode())
    return client_creds_b64.decode()

  def get_token_header(self):
    client_creds_b64 = self.get_client_credentials()
    return  {
    "Authorization" : f"Basic {client_creds_b64}" 
    }  

  def get_token_data(self):
    return  {
    "grant_type" : "client_credentials"
    }  

  def perform_auth(self):
    token_url = self.token_url
    token_data = self.get_token_data()
    token_header = self.get_token_header()
    r = requests.post(token_url, data=token_data, headers=token_header)
    
    #valid_request = r.status_code in range(200, 299):

    if r.status_code not in range(200, 299):
      return False
    data = r.json()
    now = datetime.datetime.now()
    access_token = data['access_token']
    expires_in = data['expires_in'] #seconds
    expires = now + datetime.timedelta(seconds=expires_in)
    self.access_token = access_token
    self.access_token_expires = expires
    self.access_token_did_expire = expires < now
    return True


In [55]:
spotify = SpotifyAPI(client_id, client_secret)


In [ ]:
spotify.perform_auth()

In [57]:
access_token = spotify.access_token

In [ ]:
headers = {
    "Authorization" : f"Bearer {access_token}"
}
endpoint = "https://api.spotify.com/v1/search"
data = urlencode({"q" : "Time", "type" : "track"})
print(data)

lookup_url = f"{endpoint}?{data}"
print(lookup_url)
r = requests.get(lookup_url, headers=headers)
print(r.status_code)

In [ ]:
r.json()

In [ ]:
data = urlencode({"q" : "A Lennister Always pays his debts", "type" : "track"})

lookup_url = f"{endpoint}?{data}"
r = requests.get(lookup_url, headers=headers)
r.json()